
# Start a Business in the US fastest growing city-San Antonio, TX
San Antonio, TX enjoys the biggest growth in population in from 2016 to 2017. This provides business opportunities especially for entrepreneurs who want to open their business in San Antonio.

## The goal of this project is the following to identify which business opportunity(ies) would be the best to pursue based on the other big cities.
 

Reference:

https://www.forbes.com/sites/candaceevans/2018/07/13/largest-population-boom-in-the-country-is-in-san-antonio-texas-because-business/#71b491b72b4b

## Datasets
### San Antonio, TX Zipcode and population data: 
    https://github.com/cinderjohn/IBMCapStone/blob/master/San%20Antonio.csv
    
### Phoenix, AZ Zipcode and population data: 
    https://github.com/cinderjohn/IBMCapStone/blob/master/Phoenix.csv
    
### Coordinates of All US Zipcodes:
    https://github.com/cinderjohn/IBMCapStone/blob/master/Geospatial_Coordinates.csv
    
### Foursquares venue information for San Antonio and Phoenix
     In the code
     

In [1]:
#Utility Packages
import requests
import itertools
import pandas as pd
import numpy as np
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors
import matplotlib.pyplot as plt
from matplotlib.ticker import NullFormatter

import matplotlib.ticker as ticker
from sklearn import preprocessing
!conda install -c anaconda beautifulsoup4 -y
from bs4 import BeautifulSoup
import json # library to handle JSON files

!conda install -c conda-forge geopy --yes  
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe



# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes 
#!conda install -c conda-forge/label/cf201901 folium 
import folium # map rendering library


print('Packages ready!')

Solving environment: ...working... done

## Package Plan ##

  environment location: C:\Users\house\Anaconda3

  added / updated specs: 
    - beautifulsoup4


The following packages will be UPDATED:

    beautifulsoup4:  4.6.3-py37_0         --> 4.6.3-py37_0         anaconda
    ca-certificates: 2018.03.07-0         --> 2018.03.07-0         anaconda
    certifi:         2018.11.29-py37_0    --> 2018.11.29-py37_0    anaconda
    conda:           4.5.12-py37_0        --> 4.6.2-py37_0         anaconda
    openssl:         1.1.1a-he774522_0    --> 1.1.1a-he774522_0    anaconda
    qt:              5.9.7-vc14h73c81de_0 --> 5.9.7-vc14h73c81de_0 anaconda [vc14]

Preparing transaction: ...working... done
Verifying transaction: ...working... done
Executing transaction: ...working... done




==> WARNING: A newer version of conda exists. <==
  current version: 4.5.12
  latest version: 4.6.2

Please update conda by running

    $ conda update -n base -c defaults conda




Solving environment: ...working... 
  - anaconda::ca-certificates-2018.03.07-0, anaconda::certifi-2018.11.29-py37_0, anaconda::openssl-1.1.1a-he774522_0
  - anaconda::certifi-2018.11.29-py37_0, anaconda::openssl-1.1.1a-he774522_0, defaults::ca-certificates-2018.03.07-0
  - anaconda::ca-certificates-2018.03.07-0, anaconda::certifi-2018.11.29-py37_0, defaults::openssl-1.1.1a-he774522_0
  - anaconda::certifi-2018.11.29-py37_0, defaults::ca-certificates-2018.03.07-0, defaults::openssl-1.1.1a-he774522_0
  - anaconda::ca-certificates-2018.03.07-0, defaults::certifi-2018.11.29-py37_0, defaults::openssl-1.1.1a-he774522_0
  - defaults::ca-certificates-2018.03.07-0, defaults::certifi-2018.11.29-py37_0, defaults::openssl-1.1.1a-he774522_0
  - anaconda::ca-certificates-2018.03.07-0, anaconda::openssl-1.1.1a-he774522_0, defaults::certifi-2018.11.29-py37_0
  - anaconda::openssl-1.1.1a-he774522_0, defaults::ca-certificates-2018.03.07-0, defaults::certifi-2018.11.29-py37_0done

## Package Plan ##

  e

In [2]:
## Define Foursquare Credentials and Version
CLIENT_ID = 'A0BEEBP0VG14B2EDHXDUX1YYNR0O5RLUV3CNMBUVX4UBOSGZ' # your Foursquare ID
CLIENT_SECRET = '3HELNYYP3L5ESUUC5RXE2ARB1HQQSQO4UOQQN2SOZB51T1DJ' # your Foursquare Secret
VERSION = '20180604'

In [4]:
address = 'San Antonio, TX'

geolocator = Nominatim()
sa = geolocator.geocode(address)
latitude = sa.latitude
longitude = sa.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

C:\Users\house\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: DeprecationWarning: Using Nominatim with the default "geopy/1.18.1" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  This is separate from the ipykernel package so we can avoid doing imports until


The geograpical coordinate of Toronto are 29.4246002, -98.4951405.


In [5]:
## read in SA, PHX and coordinates
sa_df = pd.read_csv('https://raw.githubusercontent.com/cinderjohn/IBMCapStone/Data/San%20Antonio%20Zip.csv')
phx_df = pd.read_csv('https://raw.githubusercontent.com/cinderjohn/IBMCapStone/master/Phoenix.csv')
zip_coordinate_df = pd.read_csv('https://raw.githubusercontent.com/cinderjohn/IBMCapStone/master/US%20Zip%20Code%20Geolocations%20from%202018%20Government%20Datal.csv')
zip_coordinate_df.dtypes

ZIP      int64
LAT    float64
LNG    float64
dtype: object

In [6]:
phx_df.dtypes
sa_df.dtypes


Zip            int64
City          object
County        object
Population    object
dtype: object

In [7]:
# create map of Toronto using latitude and longitude values
map_sa = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
#for lat, lng, borough, neighborhood in zip(merged_df['Latitude'], merged_df['Longitude'], merged_df['Borough'], merged_df['Neighborhood']):
#    label = '{}, {}'.format(neighborhood, borough)
#    label = folium.Popup(label, parse_html=True)
#    folium.CircleMarker(
#        [lat, lng],
#        radius=5,
#        popup=label,
#        color='red',
#        fill=True,
#        fill_color='#2156cc',
#        fill_opacity=0.5,
#        parse_html=False).add_to(map_toronto)  
    
map_sa

In [8]:
sa_df.head()
phx_df.head()
zip_coordinate_df.head()

,ZIP,LAT,LNG
0,601,18.180555,-66.749961
1,602,18.361945,-67.175597
2,603,18.455183,-67.119887
3,606,18.158327,-66.932928
4,610,18.295366,-67.125135


In [9]:
sa_df['Population'] = sa_df['Population'].str.replace(',', '')
phx_df['Population'] = phx_df['Population'].str.replace(',', '')

In [10]:
sa_df['Population']=sa_df['Population'].apply(lambda x:float(x))
phx_df['Population']=phx_df['Population'].apply(lambda x:float(x))

In [11]:
sa_df.dtypes
phx_df.dtypes

Zip             int64
City           object
County         object
Population    float64
dtype: object

In [12]:
#Merge Coordinate with the city zip population data
sa_merged_df = pd.merge(sa_df, zip_coordinate_df,left_on='Zip',right_on='ZIP',how='left').drop('ZIP', axis=1)
phx_merged_df = pd.merge(phx_df, zip_coordinate_df,left_on='Zip',right_on='ZIP',how='left').drop('ZIP', axis=1)

In [13]:
sa_df_final = sa_merged_df[np.isfinite(sa_merged_df['LAT'])]
phx_df_final = phx_merged_df[np.isfinite(phx_merged_df['LAT'])]
sa_df_final.dtypes

Zip             int64
City           object
County         object
Population    float64
LAT           float64
LNG           float64
dtype: object

In [14]:
sa_df_final['Zip'] = sa_df_final['Zip'].apply(str)
phx_df_final['Zip'] = phx_df_final['Zip'].apply(str)

C:\Users\house\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
C:\Users\house\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [15]:
print(phx_df_final.dtypes)
phx_df_final.head()


Zip            object
City           object
County         object
Population    float64
LAT           float64
LNG           float64
dtype: object


,Zip,City,County,Population,LAT,LNG
2,85003,Phoenix,Maricopa,9369.0,33.450662,-112.078353
3,85004,Phoenix,Maricopa,4965.0,33.451567,-112.069886
5,85006,Phoenix,Maricopa,25742.0,33.465234,-112.048771
6,85007,Phoenix,Maricopa,14040.0,33.447614,-112.090883
7,85008,Phoenix,Maricopa,56145.0,33.462954,-111.985178


In [16]:
sa_df_final.columns = ['Zipcode','City','County',
                     'Population','latitude','longitude']
phx_df_final.columns = ['Zipcode','City','County',
                     'Population','latitude','longitude']
sa_df_final['Zipcode']='SA' + sa_df_final['Zipcode'].astype(str)
phx_df_final['Zipcode']='PHX' + phx_df_final['Zipcode'].astype(str)

C:\Users\house\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
C:\Users\house\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [17]:
latitude = phx_df_final.loc[5, 'latitude'] # neighborhood latitude value
longitude = phx_df_final.loc[5, 'longitude'] # neighborhood longitude value
map_phx = folium.Map(location=[latitude, longitude], zoom_start=10)
for lat, lng, borough, neighborhood in zip(phx_df_final['latitude'], phx_df_final['longitude'], phx_df_final['Zipcode'], phx_df_final['City']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='red',
        fill=True,
        fill_color='#3156cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_phx)  
    
map_phx


In [18]:
map_sa = folium.Map(location=[latitude, longitude], zoom_start=10)
for lat, lng, borough, neighborhood in zip(sa_df_final['latitude'], sa_df_final['longitude'], sa_df_final['Zipcode'], sa_df_final['City']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='red',
        fill=True,
        fill_color='#2156cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_sa)  
    
map_sa


In [19]:
merged_df=pd.concat([sa_df_final, phx_df_final])

In [20]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [21]:
sa_df_final[sa_df_final['Zipcode']=='SA78209'].index

Int64Index([8], dtype='int64')

In [22]:
neighborhood_latitude = sa_df_final.loc[8, 'latitude'] # neighborhood latitude value
neighborhood_longitude = sa_df_final.loc[8, 'longitude'] # neighborhood longitude value

neighborhood_name = sa_df_final.loc[8, 'Zipcode'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of SA78209 are 29.488906, -98.45623499999999.


In [23]:
LIMIT = 150
radius = 500
url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/explore?client_id=A0BEEBP0VG14B2EDHXDUX1YYNR0O5RLUV3CNMBUVX4UBOSGZ&client_secret=3HELNYYP3L5ESUUC5RXE2ARB1HQQSQO4UOQQN2SOZB51T1DJ&ll=33.465234,-112.048771&v=20180604&radius=500&limit=150'

In [24]:
results = requests.get(url).json()
#results

In [25]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)
# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]
print(nearby_venues.head())

                                name  categories        lat         lng
0  Ollie Vaughn's Kitchen and Bakery        Café  33.465964 -112.048779
1                        Smooth Brew        Café  33.465552 -112.049990
2                         Guanaquito  Restaurant  33.465945 -112.050643
3                     Rainbow Donuts  Donut Shop  33.465534 -112.052255
4                       La Santisima  Taco Place  33.469189 -112.047502


In [26]:
## create a function to repeat the same process to all the neighborhoods in Toronto

def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'categories']
    
    return(nearby_venues)

In [27]:
# find the venues in all SA Zipcodes
all_venue_df = getNearbyVenues(names=merged_df['Zipcode'],
                                   latitudes=merged_df['latitude'],
                                   longitudes=merged_df['longitude']
                                  )

SA78201
SA78202
SA78203
SA78204
SA78205
SA78207
SA78208
SA78209
SA78210
SA78211
SA78212
SA78213
SA78214
SA78215
SA78216
SA78217
SA78218
SA78219
SA78220
SA78221
SA78222
SA78223
SA78224
SA78225
SA78226
SA78227
SA78228
SA78229
SA78230
SA78231
SA78232
SA78233
SA78234
SA78235
SA78236
SA78237
SA78238
SA78239
SA78240
SA78242
SA78243
SA78244
SA78245
SA78247
SA78248
SA78249
SA78250
SA78251
SA78252
SA78253
SA78254
SA78255
SA78256
SA78257
SA78258
SA78259
SA78260
SA78261
SA78263
SA78264
SA78266
PHX85003
PHX85004
PHX85006
PHX85007
PHX85008
PHX85009
PHX85012
PHX85013
PHX85014
PHX85015
PHX85016
PHX85017
PHX85018
PHX85019
PHX85020
PHX85021
PHX85022
PHX85023
PHX85024
PHX85027
PHX85028
PHX85029
PHX85031
PHX85032
PHX85033
PHX85034
PHX85035
PHX85037
PHX85040
PHX85041
PHX85042
PHX85043
PHX85044
PHX85045
PHX85048
PHX85050
PHX85051
PHX85053
PHX85054
PHX85083
PHX85085
PHX85086
PHX85087


In [28]:
all_venue_df.shape
all_venue_df.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,categories
0,SA78201,29.468413,-98.528889,Jacala Mexican Restaurant,29.468267,-98.525847,Mexican Restaurant
1,SA78201,29.468413,-98.528889,La Taqueria Huentitan Jalisco,29.467041,-98.528436,Mexican Restaurant
2,SA78201,29.468413,-98.528889,Starbucks,29.466226,-98.527546,Coffee Shop
3,SA78201,29.468413,-98.528889,De Wese's Tip Top Cafe,29.470214,-98.531357,Café
4,SA78201,29.468413,-98.528889,Church's Chicken,29.466726,-98.525842,Fried Chicken Joint


In [29]:
## check how many venues were returned for each Zipcode
all_venue_df.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,categories
Neighborhood,,,,,,
PHX85003,29,29,29,29,29,29
PHX85004,72,72,72,72,72,72
PHX85006,16,16,16,16,16,16
PHX85007,8,8,8,8,8,8
PHX85008,10,10,10,10,10,10
PHX85009,2,2,2,2,2,2
PHX85012,59,59,59,59,59,59
PHX85013,21,21,21,21,21,21
PHX85014,12,12,12,12,12,12


In [30]:
## Analyze Each Neighborhood

# one hot encoding
onehot = pd.get_dummies(all_venue_df[['categories']], prefix="", prefix_sep="")
print(onehot.columns)
# add neighborhood column back to dataframe
onehot['Neighborhood'] = all_venue_df['Neighborhood'] 

# move neighborhood column to the first column
column_list = onehot.columns.tolist()
column_number = int(column_list.index('Neighborhood'))
column_list = [column_list[column_number]] + column_list[:column_number] + column_list[column_number+1:]

Index(['Accessories Store', 'Airport Lounge', 'Airport Service',
       'Airport Terminal', 'American Restaurant', 'Arcade', 'Art Gallery',
       'Art Museum', 'Arts & Crafts Store', 'Asian Restaurant',
       ...
       'Vegetarian / Vegan Restaurant', 'Video Game Store', 'Video Store',
       'Vietnamese Restaurant', 'Warehouse Store', 'Waterfront', 'Wine Bar',
       'Wings Joint', 'Women's Store', 'Yoga Studio'],
      dtype='object', length=218)


In [31]:
## group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

grouped = onehot.groupby('Neighborhood').mean().reset_index()
grouped.shape

(86, 218)

In [32]:
## put that into a pandas dataframe
## First, write a function to sort the venues in descending order.

def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]
## create the new dataframe and display the top 10 venues for each neighborhood.

num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = grouped['Neighborhood']

for ind in np.arange(grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,PHX85003,American Restaurant,Music Venue,Coffee Shop,Pub,Pizza Place,Sushi Restaurant,Bowling Alley,Building,Burger Joint,Rock Club
1,PHX85004,Coffee Shop,Pizza Place,American Restaurant,Hotel,Bar,Pub,Park,Sandwich Place,Hotel Bar,Salon / Barbershop
2,PHX85006,Mexican Restaurant,Taco Place,Café,Yoga Studio,Hobby Shop,Bike Shop,Gas Station,Fast Food Restaurant,Clothing Store,Gay Bar
3,PHX85007,Food Truck,Athletics & Sports,Chinese Restaurant,Breakfast Spot,Sandwich Place,Park,Plaza,French Restaurant,Fountain,Flower Shop
4,PHX85008,Burger Joint,Pharmacy,Fast Food Restaurant,Sandwich Place,Grocery Store,Coffee Shop,Gym,Pizza Place,French Restaurant,Pool
5,PHX85009,Mexican Restaurant,Auto Workshop,Yoga Studio,Flower Shop,Gas Station,Furniture / Home Store,Frozen Yogurt Shop,Fried Chicken Joint,French Restaurant,Fountain
6,PHX85012,Boutique,Pizza Place,Furniture / Home Store,Coffee Shop,Breakfast Spot,Deli / Bodega,Clothing Store,Record Shop,Mexican Restaurant,Candy Store
7,PHX85013,Cosmetics Shop,Park,Coffee Shop,Cocktail Bar,Furniture / Home Store,Gay Bar,Mobile Phone Shop,Sandwich Place,Taco Place,Thai Restaurant
8,PHX85014,Pizza Place,Moving Target,Arts & Crafts Store,Pet Store,Restaurant,Chinese Restaurant,Café,Sushi Restaurant,Mexican Restaurant,Dessert Shop
9,PHX85015,Record Shop,Fried Chicken Joint,Discount Store,Asian Restaurant,Mexican Restaurant,Cajun / Creole Restaurant,Flower Shop,Furniture / Home Store,Frozen Yogurt Shop,French Restaurant


In [33]:
## Cluster Neighborhoods
## Run k-means to cluster the neighborhood into 10 clusters.

# set number of clusters
kclusters = 10

grouped_clustering = grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[:].shape 

(86,)

In [34]:
kmeans

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
    n_clusters=10, n_init=10, n_jobs=None, precompute_distances='auto',
    random_state=0, tol=0.0001, verbose=0)

In [35]:
all_merged = merged_df.head(86)

In [36]:
all_merged['Cluster Labels']=kmeans.labels_
all_merged

C:\Users\house\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


,Zipcode,City,County,Population,latitude,longitude,Cluster Labels
0,SA78201,San Antonio,Bexar,45307.0,29.468413,-98.528889,3
1,SA78202,San Antonio,Bexar,11691.0,29.428169,-98.460881,3
2,SA78203,San Antonio,Bexar,6099.0,29.415100,-98.459123,3
3,SA78204,San Antonio,Bexar,11125.0,29.404404,-98.505028,3
4,SA78205,San Antonio,Bexar,1453.0,29.423945,-98.486420,3
6,SA78207,San Antonio,Bexar,55514.0,29.422124,-98.525978,3
7,SA78208,San Antonio,Bexar,3736.0,29.439819,-98.458549,3
8,SA78209,San Antonio,Bexar,39197.0,29.488906,-98.456235,3
9,SA78210,San Antonio,Bexar,36865.0,29.395776,-98.464401,3
10,SA78211,San Antonio,Bexar,31944.0,29.342550,-98.570024,3


In [37]:
# merge grouped with toronto_data to add latitude/longitude for each neighborhood
neighborhoods_venues_sorted=neighborhoods_venues_sorted.rename(index=str, columns={"Neighborhood": "Zipcode"})
all_merged = all_merged.join(neighborhoods_venues_sorted.set_index('Zipcode'), on='Zipcode')

In [38]:
all_merged

,Zipcode,City,County,Population,latitude,longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,SA78201,San Antonio,Bexar,45307.0,29.468413,-98.528889,3,Mexican Restaurant,Fried Chicken Joint,Chinese Restaurant,Coffee Shop,Food Truck,Bus Station,Café,Grocery Store,Fast Food Restaurant,Pawn Shop
1,SA78202,San Antonio,Bexar,11691.0,29.428169,-98.460881,3,Discount Store,Mexican Restaurant,BBQ Joint,Fast Food Restaurant,Mobile Phone Shop,Grocery Store,Chinese Restaurant,Nail Salon,Pizza Place,Pharmacy
2,SA78203,San Antonio,Bexar,6099.0,29.415100,-98.459123,3,Southern / Soul Food Restaurant,Fried Chicken Joint,Mexican Restaurant,Gym / Fitness Center,Historic Site,Bakery,Yoga Studio,Flower Shop,Frozen Yogurt Shop,French Restaurant
3,SA78204,San Antonio,Bexar,11125.0,29.404404,-98.505028,3,Bakery,Art Gallery,Lounge,Bar,Mexican Restaurant,Brewery,Pizza Place,Recreation Center,Frozen Yogurt Shop,Fried Chicken Joint
4,SA78205,San Antonio,Bexar,1453.0,29.423945,-98.486420,3,Hotel,Steakhouse,Bar,Theater,Seafood Restaurant,Ice Cream Shop,Historic Site,History Museum,Dessert Shop,Plaza
6,SA78207,San Antonio,Bexar,55514.0,29.422124,-98.525978,3,Fast Food Restaurant,Mexican Restaurant,Fried Chicken Joint,Pizza Place,Video Store,Flea Market,Furniture / Home Store,Frozen Yogurt Shop,French Restaurant,Fountain
7,SA78208,San Antonio,Bexar,3736.0,29.439819,-98.458549,3,Fast Food Restaurant,Hotel,BBQ Joint,Dive Bar,Gas Station,Cajun / Creole Restaurant,Mexican Restaurant,Chinese Restaurant,Convenience Store,Burger Joint
8,SA78209,San Antonio,Bexar,39197.0,29.488906,-98.456235,3,Pizza Place,Ice Cream Shop,Art Museum,Gym,Pharmacy,Pet Store,Clothing Store,Salon / Barbershop,Organic Grocery,Café
9,SA78210,San Antonio,Bexar,36865.0,29.395776,-98.464401,3,Buffet,Yoga Studio,Flower Shop,Gas Station,Furniture / Home Store,Frozen Yogurt Shop,Fried Chicken Joint,French Restaurant,Fountain,Food Truck
10,SA78211,San Antonio,Bexar,31944.0,29.342550,-98.570024,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [39]:
all_merged.sort_values(['Cluster Labels','City', 'Zipcode', '1st Most Common Venue','2nd Most Common Venue'])

,Zipcode,City,County,Population,latitude,longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
30,PHX85032,Phoenix,Maricopa,65485.0,33.625920,-112.002503,0,Park,Intersection,Playground,Flea Market,Furniture / Home Store,Frozen Yogurt Shop,Fried Chicken Joint,French Restaurant,Fountain,Food Truck
39,SA78240,San Antonio,Bexar,51111.0,29.525020,-98.606564,0,Grocery Store,Liquor Store,Pizza Place,Salon / Barbershop,Mobile Phone Shop,Flea Market,Frozen Yogurt Shop,Fried Chicken Joint,French Restaurant,Fountain
28,SA78229,San Antonio,Bexar,28804.0,29.501861,-98.569302,1,Hotel,Sandwich Place,Yoga Studio,Fast Food Restaurant,Furniture / Home Store,Frozen Yogurt Shop,Fried Chicken Joint,French Restaurant,Fountain,Food Truck
38,SA78239,San Antonio,Bexar,28736.0,29.518637,-98.362961,1,IT Services,Tree,Yoga Studio,Fast Food Restaurant,Furniture / Home Store,Frozen Yogurt Shop,Fried Chicken Joint,French Restaurant,Fountain,Food Truck
19,PHX85020,Phoenix,Maricopa,32850.0,33.567351,-112.053591,2,Mexican Restaurant,Bowling Alley,Park,General Entertainment,Gas Station,Furniture / Home Store,Frozen Yogurt Shop,Fried Chicken Joint,French Restaurant,Fountain
31,PHX85033,Phoenix,Maricopa,53037.0,33.492969,-112.209757,2,Bus Station,Yoga Studio,Flower Shop,Gas Station,Furniture / Home Store,Frozen Yogurt Shop,Fried Chicken Joint,French Restaurant,Fountain,Food Truck
2,PHX85003,Phoenix,Maricopa,9369.0,33.450662,-112.078353,3,American Restaurant,Music Venue,Coffee Shop,Pub,Pizza Place,Sushi Restaurant,Bowling Alley,Building,Burger Joint,Rock Club
3,PHX85004,Phoenix,Maricopa,4965.0,33.451567,-112.069886,3,Coffee Shop,Pizza Place,American Restaurant,Hotel,Bar,Pub,Park,Sandwich Place,Hotel Bar,Salon / Barbershop
5,PHX85006,Phoenix,Maricopa,25742.0,33.465234,-112.048771,3,Mexican Restaurant,Taco Place,Café,Yoga Studio,Hobby Shop,Bike Shop,Gas Station,Fast Food Restaurant,Clothing Store,Gay Bar
6,PHX85007,Phoenix,Maricopa,14040.0,33.447614,-112.090883,3,Food Truck,Athletics & Sports,Chinese Restaurant,Breakfast Spot,Sandwich Place,Park,Plaza,French Restaurant,Fountain,Flower Shop


In [40]:
all_merged.to_excel("C:/Users/house/Documents/SA_PHX_compare.xlsx") 


In [ ]:
#The project token is an authorization token that is used to access project resources like data sources, connections, and used by platform APIs.
#from project_lib import Project
#project = Project(None, 'Capstone','p-bb84538548e5617dea7c8536cefe1485f6da66b3')
#pc = project.project_context
#project.save_data(data=all_merged.to_csv(index=False),file_name='SA_PHX_compare.csv',overwrite=True)